## Preliminaries

### The most important Jupyter notebook tricks
 - Press Esc to escape edit mode and add / remove cells etc.
 - When not in Edit mode: A inserts a cell above, B below
 - When not in Edit mode: DD deletes a cell
 - When in Edit mode: Shift + Enter executes the cell
 - Interrupt a hung kernel with the Stop button
 - Or restart under Kernel -> Restart

### The most important bash tricks

In [5]:
# pipe output to the next command with |
# provides the seed as input to the "mnemonic-new command"
bx seed | bx mnemonic-new

# capture output in variables with var=$(...). Notice the missing spaces around =
var=$(bx seed)

# later refer to the variable with $. echo prints stuff
echo $var

over hole smart blind speak number route topple embrace math develop advance radio oyster voyage word panic puzzle
f949a33e0df3c43e1c7ba3785f73ab6b5961a062bb2d2f3a


### Explore the bx tool

In [158]:
bx help


Usage: bx COMMAND [--help]

Version: 3.2.0

Info: The bx commands are:

address-decode
address-embed
address-encode
base16-decode
base16-encode
base58-decode
base58-encode
base58check-decode
base58check-encode
base64-decode
base64-encode
bitcoin160
bitcoin256
btc-to-satoshi
cert-new
cert-public
ec-add
ec-add-secrets
ec-multiply
ec-multiply-secrets
ec-new
ec-to-address
ec-to-ek
ec-to-public
ec-to-wif
ek-address
ek-new
ek-public
ek-public-to-address
ek-public-to-ec
ek-to-address
ek-to-ec
fetch-balance
fetch-header
fetch-height
fetch-history
fetch-public-key
fetch-stealth
fetch-tx
fetch-tx-index
fetch-utxo
hd-new
hd-private
hd-public
hd-to-ec
hd-to-public
help
input-set
input-sign
input-validate
message-sign
message-validate
mnemonic-new
mnemonic-to-seed
qrcode
ripemd160
satoshi-to-btc
script-decode
script-encode
script-to-address
seed
send-tx
send-tx-node
send-tx-p2p
settings
sha160
sha256
sha512
stealth-decode
stealth-encode
stealth-public
stealth-secret
stealth-shared
token-new
tx-dec

In [13]:
bx help base16-decode 


Usage: bx base16-decode [-h] [--config value] [BASE16]                   

Info: Convert a Base16 value to binary data.                             

Options (named):

-c [--config]        The path to the configuration settings file.        
-h [--help]          Get a description and instructions for this command.

Arguments (positional):

BASE16               The Base16 value to decode as binary data. If not   
                     specified the value is read from STDIN.             


### Ensure testnet is configured

In [2]:
export BX_CONFIG="../bx_config_files/bx_testnet.cfg"
bx fetch-height

2105099


# Transaction Build - BX Example
In this BX tutorial, we construct a transaction which spends one P2PKH output to three (not six) other outputs.

<img src="images/first_transaction.jpg" alt="drawing" style="" width="700px"/>



##  1. Locate your UTXO
A testnet address has been funded for you. Use the `sha256` value of your email address as a private key and derive the corresponding public key, as well as testnet P2PKH address. Find the UTXO that is locked to that P2PKH output script.

In [2]:
# Find your address
# Useful bx functions: ec-to-public, ripemd160, ec-to-address
email="cercatrova21@protonmail.com"
privkey=$(echo -n $email | sha256sum | awk '{print $1}')
pubkey=$(echo $privkey | bx ec-to-public)
pubkeyhash=$(echo $pubkey | bx sha256 | bx ripemd160)
address=$(echo $pubkey | bx ec-to-address)
echo $privkey
echo $pubkey
echo $address
echo $pubkeyhash

d310a5ef76dd2e0cbbda5213a2093cff728a725b5c5b2bee445c7fc8d35b4473
0246dfa9a04864d88790b0ed70b987c23fa92a2bfe442197b8f5cb54564fcee6bf
mrqnmMH1msoB3B7QMi3X8D85tipDWS7aUy
7c395509796f7014290139c9d561ef84bb3b4d3b


In [3]:
my_mnemonic="apology fix coast coil reunion couch idle off fiscal horror coffee deposit defy swift bless alley print sign"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)
echo $my_mnemonic
echo $hd_master_private
echo $hd_master_public

apology fix coast coil reunion couch idle off fiscal horror coffee deposit defy swift bless alley print sign
tprv8ZgxMBicQKsPdHcXbFm8KUfEuox1eHFSePzQkzAvx865deP7vEdMjS8TJdD8RFsW8Vgj5CWVMnZbTVEw3J2GSXbHeRQavRyjD91geTrogPD
tpubD6NzVbkrYhZ4WkeKUuRiitKMUqTwocSMDhbC3WDENPtUU8dtYdSwuvkKUnDEzAsNnYyxK2CADNNbyG9jGnpcWk6iKbKvKzL9auz5jAF29Rc


In [67]:
hd_m_44h_1h_1h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0)

hd_m_44h_1h_1h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx sha256 | bx ripemd160)
address_m_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx ec-to-address)
privkey_m_44h_1h_1h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0_0 | bx hd-to-ec)
pubkey_m_44h_1h_1h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx ec-to-public)


hd_m_44h_1h_1h_0_1=$(bx hd-private --index 1 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx sha256 | bx ripemd160)
address_m_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx ec-to-address)
privkey_m_44h_1h_1h_0_1=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0_1 | bx hd-to-ec)
pubkey_m_44h_1h_1h_0_1=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx ec-to-public)

hd_m_44h_1h_1h_0_2=$(bx hd-private --index 2 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx sha256 | bx ripemd160)
address_m_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx ec-to-address)
privkey_m_44h_1h_1h_0_2=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0_2 | bx hd-to-ec)
pubkey_m_44h_1h_1h_0_2=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx ec-to-public)

echo $address_m_44h_1h_1h_0_0
echo $address_m_44h_1h_1h_0_1
echo $address_m_44h_1h_1h_0_2

echo $privkey_m_44h_1h_1h_0_0
echo $pubkey_m_44h_1h_1h_0_0

mj3gp3BZs8Cyst3rLDoh3hf7edttTBJDhZ
n3vGNDJxYsu8kgVjStYrcf7NoCygNTFXnj
mrLbFKcbuZ37t3js3e89fNgLRizB3xKRoX
1c5cc83e53526592504bce5af82406d0a9340e3ac5182712616da33a7c4d6ff8
028ad459258584b2b34cb2a2498528180de6ae548d77ba79b8c7e696a727bfe061


In [11]:
echo $address_m_44h_1h_1h_0_0 | bx fetch-history --format json

{
    "transfers": [
        {
            "received": {
                "hash": "a954cf9e01a3d698086f5ff8c46b01a54c755d803b8496069f993ff7f2e3c900",
                "height": "2105030",
                "index": "0"
            },
            "value": "2833"
        }
    ]
}


In [12]:
# Save variables for later when we construct the transaction
previous_txid=$(echo $address_m_44h_1h_1h_0_0 | bx fetch-history --format json | jq -r ".transfers[0].received.hash")
echo $previous_txid
previous_output_index=$(echo $address_m_44h_1h_1h_0_0 | bx fetch-history --format json | jq -r ".transfers[0].received.index")
echo $previous_output_index
previous_amount=$(echo $address_m_44h_1h_1h_0_0 | bx fetch-history --format json | jq -r ".transfers[0].value")
echo $previous_amount

a954cf9e01a3d698086f5ff8c46b01a54c755d803b8496069f993ff7f2e3c900
0
2833


<hr style="border: 0.5px dashed #000;">

## 1. Setup receiving wallet.


### Create new mnemonic and derive hd master keys.

In [9]:
my_mnemonic=$(bx seed | bx mnemonic-new)
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)
echo $my_mnemonic
echo $hd_master_private
echo $hd_master_public

apology fix coast coil reunion couch idle off fiscal horror coffee deposit defy swift bless alley print sign
tprv8ZgxMBicQKsPdHcXbFm8KUfEuox1eHFSePzQkzAvx865deP7vEdMjS8TJdD8RFsW8Vgj5CWVMnZbTVEw3J2GSXbHeRQavRyjD91geTrogPD
tpubD6NzVbkrYhZ4WkeKUuRiitKMUqTwocSMDhbC3WDENPtUU8dtYdSwuvkKUnDEzAsNnYyxK2CADNNbyG9jGnpcWk6iKbKvKzL9auz5jAF29Rc


### Account 1: Generate destination key pairs.

You will spend the previously received utxo to receiving addresses from `account 1`, namely `/44'/1'/1'/0/0`, `/44'/1'/1'/0/1`, `/44'/1'/1'/0/2`, `/44'/1'/1'/0/3` ...


In [10]:
# Useful bx functions: hd-private, hd-to-public, sha256, ripemd160
# Account 1 receiving parent: m/44'/1'/1'/0
hd_m_44h_1h_1h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0)

hd_m_44h_1h_1h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_0=$(bx hd-to-public $hd_m_44h_1h_1h_0_0 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_1=$(bx hd-private --index 1 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_2=$(bx hd-private --index 2 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx sha256 | bx ripemd160)

echo $publickeyhash_44h_1h_1h_0_0
echo $publickeyhash_44h_1h_1h_0_1
echo $publickeyhash_44h_1h_1h_0_2

26b858fb6e462af5ee49a33694578b4c01736892
f5bb6b7c5525becf565b0204e9546a99e92becec
76b3a79e83fc517d460440d870242b5a4b1adeb9


In [52]:
hd_m_44h_1h_1h_0_3=$(bx hd-private --index 2 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_3=$(bx hd-to-public $hd_m_44h_1h_1h_0_3 | bx hd-to-ec | bx sha256 | bx ripemd160)
echo $publickeyhash_44h_1h_1h_0_3

76b3a79e83fc517d460440d870242b5a4b1adeb9


## 2. Spending a P2PKH output.

### Create output scripts.

* **Compose P2PKH(destination public key hash) output scripts for each destination.**

In [77]:
# Useful bx functions: script-encode
output_script1=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_0"] EQUALVERIFY CHECKSIG")
output_script2=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_1"] EQUALVERIFY CHECKSIG")
output_script3=$(bx script-encode "DUP HASH160 ["7c395509796f7014290139c9d561ef84bb3b4d3b"] EQUALVERIFY CHECKSIG")
echo $output_script1
echo $output_script2
echo $output_script3


76a91426b858fb6e462af5ee49a33694578b4c0173689288ac
76a914f5bb6b7c5525becf565b0204e9546a99e92becec88ac
76a9147c395509796f7014290139c9d561ef84bb3b4d3b88ac


In [85]:
# Useful bx functions: tx-encode
single_output_amount=$(expr 6000)
echo $single_output_amount



6000


### Construct transaction template for signing.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [41]:
my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script0:$single_output_amount \
--output $output_script1:$single_output_amount \
--output $output_script2:$single_output_amount)
echo $my_tx

010000000100c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a90000000000ffffffff038d070000000000001976a91426b858fb6e462af5ee49a33694578b4c0173689288ac8d070000000000001976a914f5bb6b7c5525becf565b0204e9546a99e92becec88ac8d070000000000001976a91476b3a79e83fc517d460440d870242b5a4b1adeb988ac00000000


In [54]:
my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script3:$single_output_amount)
echo $my_tx

010000000100c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a90000000000ffffffff018d070000000000001976a91476b3a79e83fc517d460440d870242b5a4b1adeb988ac00000000


In [86]:
my_tx=$(bx tx-encode \
--input $previous_txid:0:4294967295 \
--input $previous_txid:1:4294967295 \
--input $previous_txid:2:4294967295 \
--output $output_script3:$single_output_amount)
echo $my_tx

010000000300c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a90000000000ffffffff00c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a90100000000ffffffff00c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a90200000000ffffffff0170170000000000001976a9147c395509796f7014290139c9d561ef84bb3b4d3b88ac00000000


### Sign transaction.

**Fetch previous output script (check output index ! )**

In [55]:
# Useful bx functions: fetch-tx
bx fetch-tx $previous_txid

transaction
{
    hash a954cf9e01a3d698086f5ff8c46b01a54c755d803b8496069f993ff7f2e3c900
    inputs
    {
        input
        {
            address_hash 7c395509796f7014290139c9d561ef84bb3b4d3b
            previous_output
            {
                hash 0625d7e116e6b2ce8909a3a68bf7b446205c1bc4863e0ec7a83fe57dd4198c45
                index 1
            }
            script "[3045022100f9c97477b6a967ec567ffc960d7f530347a4f42abf6d0a3e5cf1b7cda14d5bfb022043893690d3c83d7b463dbec039c9e36964aaeeedb565e3234334697056fe9a9d01] [0246dfa9a04864d88790b0ed70b987c23fa92a2bfe442197b8f5cb54564fcee6bf]"
            sequence 4294967295
        }
    }
    lock_time 0
    outputs
    {
        output
        {
            address_hash 26b858fb6e462af5ee49a33694578b4c01736892
            script "dup hash160 [26b858fb6e462af5ee49a33694578b4c01736892] equalverify checksig"
            value 2833
        }
        output
        {
            address_hash f5bb6b7c5525becf565b0204e9546a99e92becec
        

In [87]:
previous_output_script1=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[0].script")
previous_output_script2=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[1].script")
previous_output_script3=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[2].script")

echo $previous_output_script1
echo $previous_output_script2
echo $previous_output_script3

dup hash160 [26b858fb6e462af5ee49a33694578b4c01736892] equalverify checksig
dup hash160 [f5bb6b7c5525becf565b0204e9546a99e92becec] equalverify checksig
dup hash160 [76b3a79e83fc517d460440d870242b5a4b1adeb9] equalverify checksig


**Sign transaction with previous output script.**

In [88]:
# Useful bx functions: input-sign
signature1=$(bx input-sign --sign_type all --index 0 $privkey_m_44h_1h_1h_0_0 "$previous_output_script1" $my_tx)
signature2=$(bx input-sign --sign_type all --index 1 $privkey_m_44h_1h_1h_0_1 "$previous_output_script2" $my_tx)
signature3=$(bx input-sign --sign_type all --index 2 $privkey_m_44h_1h_1h_0_2 "$previous_output_script3" $my_tx)
#echo $my_tx
echo $signature1
echo $signature2
echo $signature3

304402200f980ffde703879457866a25ec501be5538d6ebcdb010e1b596014c072912f3e022058e3b16d5b50f9b5f51299a05848dfcb6e61dd7884ca63f4506008842fdfded601
3044022024e1f5923706b4fdd3bfddd85ba0fccef0ee8b73b711ee88d0c212e86232606a022028af4b417f39fe3229732cc9f780c1db02abba3995680cbc5993b01ae151f3ad01
304402207206e641572a97ef66134a99f4abc385a4c683f17a6f6b3053b03de7afa81846022058e7a80ecc883578c172095a85227d6e77e4dd01beaf8e3393429ec62a6d361c01


**Set the input script into the finalised transaction.**

In [89]:
# Useful bx functions: input-set
my_tx=$(bx input-set --index 0 "[$signature1] [$pubkey_m_44h_1h_1h_0_0]" $my_tx)
my_tx=$(bx input-set --index 1 "[$signature2] [$pubkey_m_44h_1h_1h_0_1]" $my_tx)
my_tx=$(bx input-set --index 2 "[$signature3] [$pubkey_m_44h_1h_1h_0_2]" $my_tx)
echo $my_tx

010000000300c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a9000000006a47304402200f980ffde703879457866a25ec501be5538d6ebcdb010e1b596014c072912f3e022058e3b16d5b50f9b5f51299a05848dfcb6e61dd7884ca63f4506008842fdfded60121028ad459258584b2b34cb2a2498528180de6ae548d77ba79b8c7e696a727bfe061ffffffff00c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a9010000006a473044022024e1f5923706b4fdd3bfddd85ba0fccef0ee8b73b711ee88d0c212e86232606a022028af4b417f39fe3229732cc9f780c1db02abba3995680cbc5993b01ae151f3ad012102733dfd6510b3ef945902ba1419ed5cbb5b79994dedc96a96c9e424556554519effffffff00c9e3f2f73f999f0696843b805d754ca5016bc4f85f6f0898d6a3019ecf54a9020000006a47304402207206e641572a97ef66134a99f4abc385a4c683f17a6f6b3053b03de7afa81846022058e7a80ecc883578c172095a85227d6e77e4dd01beaf8e3393429ec62a6d361c012103866447d2e4a357e7b3f563c8550af1e83e5d85d0c502bb29f5ae85b7920b6fd2ffffffff0170170000000000001976a9147c395509796f7014290139c9d561ef84bb3b4d3b88ac00000000


## 3. Validate & Broadcast the endorsed transaction

In [90]:
bx validate-tx $my_tx

invalid script


: 255

In [91]:
bx send-tx $my_tx

invalid script


: 255

## Check history for sending and a receiving address

In [ ]:
# Useful bx functions: fetch-history